In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import src.config as config

from datetime import datetime

from src.inference import (
    get_feature_store, 
    load_batch_of_features_for_inference,
    load_model_from_registry,
    get_model_predictions,
    )

2025-03-21 14:29:51,991 WARNING: FutureIncompatibilityWarning: 

This is a future version incompatibility warning from Holidays v0.60
to inform you about an upcoming change in our API versioning strategy that may affect your
project's dependencies. Starting from version 1.0 onwards, we will be following a loose form of
Semantic Versioning (SemVer, https://semver.org) to provide clearer communication regarding any
potential breaking changes.

This means that while we strive to maintain backward compatibility, there might be occasional
updates that introduce breaking changes to our API. To ensure the stability of your projects,
we highly recommend pinning the version of our API that you rely on. You can pin your current
holidays v0.x dependency (e.g., holidays==0.60) or limit it (e.g., holidays<1.0) in order to
avoid potentially unwanted upgrade to the version 1.0 when it's released (ETA 2025Q1-Q2).

If you have any questions or concerns regarding this change, please don't hesitate to re

In [3]:
current_date = datetime.now().date()

data = load_batch_of_features_for_inference(current_date)
display(data.head())
display(data.tail())

2025-03-21 14:29:54,880 INFO: Initializing external client
2025-03-21 14:29:54,881 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-21 14:29:56,507 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1051798
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.43s) 


,datetime,demand,ba_code
17406,2024-03-07,51039.0,AECI
4513,2024-03-08,52012.0,AECI
7844,2024-03-09,56646.0,AECI
10221,2024-03-10,54399.0,AECI
17068,2024-03-11,53728.0,AECI


,datetime,demand,ba_code
25033,2025-03-15,2053.0,WAUW
25090,2025-03-16,1965.0,WAUW
25189,2025-03-17,1946.0,WAUW
25177,2025-03-18,2006.0,WAUW
25140,2025-03-19,2158.0,WAUW


In [4]:
model, preprocessing_pipeline = load_model_from_registry()

2025-03-21 14:30:20,072 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-21 14:30:20,076 INFO: Initializing external client
2025-03-21 14:30:20,076 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-21 14:30:21,267 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1051798


In [10]:
# The features end yesterday
features_end = str(data["datetime"].dt.date.max())
print(features_end)

predictions = get_model_predictions(
    model=model, 
    preprocessing_pipeline=preprocessing_pipeline, 
    X=data, 
    features_end=features_end,
    )
predictions.head()

2025-03-19


,ba_code,predicted_demand,datetime
0,AECI,61723.0,2025-03-20
1,AVA,33361.0,2025-03-20
2,AZPS,67868.0,2025-03-20
3,BANC,44702.0,2025-03-20
4,BPAT,143097.0,2025-03-20


In [ ]:
feature_store = get_feature_store()

feature_group_predictions = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=config.FEATURE_GROUP_MODEL_PREDICTIONS_VERSION,
    description="Predictions generated by the model",
    primary_key=["ba_code", "datetime"],
    event_time="datetime",
)

In [155]:
feature_group_predictions.insert(predictions, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1051798/fs/1043525/fg/1420834


Uploading Dataframe: 100.00% |██████████| Rows 53/53 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: model_predictions_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1051798/jobs/named/model_predictions_feature_group_1_offline_fg_materialization/executions


(Job('model_predictions_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)